<a href="https://colab.research.google.com/github/shubhamgundawarNYU/Big-Data-Project-Group-16/blob/main/misc-datasets-notebooks/NYPD_Criminal_Court_Summons_(Historic).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**BIG DATA PROJECT**

### NYPD Criminal Court Summons (Historic) 
Link to Dataset (https://data.cityofnewyork.us/Public-Safety/NYPD-Criminal-Court-Summons-Historic-/sv2w-rv3k)

### DATA CLEANING AT SCALE

#### Mounting Google Drive to Google Collab Notebook to Load the Data Set

Make sure you have the dataset in your Google Drive and you mount your drive to the Colab.

The file should be at the following path: `gdrive/My Drive/NYPD_Complaint_Data_Historic.csv`


In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#### Importing required and Necessary Libraries for cleaning the data present in the data set

In [ ]:
import numpy as np
import pandas as pd
import io

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 32 kB/s 
     |████████████████████████████████| 198 kB 49.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=8cd5e0bd0694e717d2ac34e8cfdee16fcab44dcc7c1d17152d323ad6e550505d
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark



# **Running Pyspark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 2.3.2 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab. One important note is that if you are new in Spark, it is better to avoid Spark 2.4.0 version since some people have already complained about its compatibility issue with python. 
Follow the steps to install the dependencies:

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

Now that you installed Spark and Java in Colab, it is time to set the environment path which enables you to run Pyspark in your Colab environment. Set the location of Java and Spark by running the following code:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

Run a local spark session to test your installation:

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()

#### Reading the Data Set CSV File using `spark.read.csv()` Function

In [ ]:
df = spark.read.csv("/content/gdrive/MyDrive/Big Data/NYPD_Criminal_Court_Summons__Historic_.csv", inferSchema=True, header =True)

In [ ]:
df.count()

5336264

#### Get Data Type for each column present in the Data Set




In [ ]:
df.printSchema()

root
 |-- SUMMONS_KEY: integer (nullable = true)
 |-- SUMMONS_DATE: string (nullable = true)
 |-- OFFENSE_DESCRIPTION: string (nullable = true)
 |-- LAW_SECTION_NUMBER: string (nullable = true)
 |-- LAW_DESCRIPTION: string (nullable = true)
 |-- SUMMONS_CATEGORY_TYPE: string (nullable = true)
 |-- AGE_GROUP: string (nullable = true)
 |-- SEX: string (nullable = true)
 |-- RACE: string (nullable = true)
 |-- JURISDICTION_CODE: string (nullable = true)
 |-- BORO: string (nullable = true)
 |-- PRECINCT_OF_OCCUR: string (nullable = true)
 |-- X_COORDINATE_CD: string (nullable = true)
 |-- Y_COORDINATE_CD: integer (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Lon_Lat: string (nullable = true)



#### Outputing the List of Columns in the Data Set

In [ ]:
df.columns

['SUMMONS_KEY',
 'SUMMONS_DATE',
 'OFFENSE_DESCRIPTION',
 'LAW_SECTION_NUMBER',
 'LAW_DESCRIPTION',
 'SUMMONS_CATEGORY_TYPE',
 'AGE_GROUP',
 'SEX',
 'RACE',
 'JURISDICTION_CODE',
 'BORO',
 'PRECINCT_OF_OCCUR',
 'X_COORDINATE_CD',
 'Y_COORDINATE_CD',
 'Latitude',
 'Longitude',
 'Lon_Lat']

#### Get top 10 rows of the complaints dataframe

In [ ]:
df.show(n=10)

+-----------+------------+--------------------+------------------+---------------+---------------------+---------+----+--------------+-----------------+--------+-----------------+---------------+---------------+------------------+------------------+--------------------+
|SUMMONS_KEY|SUMMONS_DATE| OFFENSE_DESCRIPTION|LAW_SECTION_NUMBER|LAW_DESCRIPTION|SUMMONS_CATEGORY_TYPE|AGE_GROUP| SEX|          RACE|JURISDICTION_CODE|    BORO|PRECINCT_OF_OCCUR|X_COORDINATE_CD|Y_COORDINATE_CD|          Latitude|         Longitude|             Lon_Lat|
+-----------+------------+--------------------+------------------+---------------+---------------------+---------+----+--------------+-----------------+--------+-----------------+---------------+---------------+------------------+------------------+--------------------+
|  192724412|  01/24/2019|   UNINSURED VEHICLE|               319|            VTL|                  VTL|  UNKNOWN|null|          null|                0|NEW YORK|               18|        

## We see that the columns `X_COORD_CD`,`Y_COORD_CD`,`Latitude`, `Longitude` and `Lat_Lon` conveys the same data as `New Georeferenced Column`.

#### Hence, we drop those columns and keep only `New Georeferenced Column` column in our cleaned dataset.

In [ ]:
df = df.drop('X_COORDINATE_CD','Y_COORDINATE_CD','Latitude','Longitude','Lon_Lat')

In [ ]:
df.columns

['SUMMONS_KEY',
 'SUMMONS_DATE',
 'OFFENSE_DESCRIPTION',
 'LAW_SECTION_NUMBER',
 'LAW_DESCRIPTION',
 'SUMMONS_CATEGORY_TYPE',
 'AGE_GROUP',
 'SEX',
 'RACE',
 'JURISDICTION_CODE',
 'BORO',
 'PRECINCT_OF_OCCUR']

#### Removing all the **duplicate** entries

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.count()

5336264

In [ ]:
df.distinct().count()

5336264

#### **Checking** if the SUMMONS_KEY is unique or not

In [ ]:
df.select('SUMMONS_KEY').distinct().count()

5336248

#### As we can see `SUMMONS_KEY` should have been unique, but it is not.
#### Let's see what are the duplicate values.

In [ ]:
df1 = df.groupBy('SUMMONS_KEY').count().filter("count > 1")
df1.drop('count').count()

12

In [ ]:
df1.sort('SUMMONS_KEY').show(n = 10)

+-----------+-----+
|SUMMONS_KEY|count|
+-----------+-----+
|    5542337|    2|
|   85674056|    3|
|   90228979|    2|
|  109595661|    2|
|  148527446|    3|
|  148918930|    2|
|  148968083|    3|
|  157849320|    2|
|  162208010|    2|
|  163839842|    2|
+-----------+-----+
only showing top 10 rows



#### Check for complaint number `100509703`

In [ ]:
df.filter('SUMMONS_KEY = 162208010').show()

+-----------+------------+-------------------+------------------+-------------------+---------------------+---------+---+-------+-----------------+--------+-----------------+
|SUMMONS_KEY|SUMMONS_DATE|OFFENSE_DESCRIPTION|LAW_SECTION_NUMBER|    LAW_DESCRIPTION|SUMMONS_CATEGORY_TYPE|AGE_GROUP|SEX|   RACE|JURISDICTION_CODE|    BORO|PRECINCT_OF_OCCUR|
+-----------+------------+-------------------+------------------+-------------------+---------------------+---------+---+-------+-----------------+--------+-----------------+
|  162208010|  08/04/2018|     ACCEPT ON HAIL|            19-507|Administrative Code|                  TLC|    18-24|  F|  BLACK|                0|BROOKLYN|               70|
|  162208010|  08/04/2018|     ACCEPT ON HAIL|            19-507|Administrative Code|                  TLC|  UNKNOWN|  F|UNKNOWN|                0|BROOKLYN|               70|
+-----------+------------+-------------------+------------------+-------------------+---------------------+---------+---+----

#### We understand, that complaint number is not specifically unique. The dataset has rows with duplicated complaint numbers having unique information for all other columns. Thus, we cannot drop the entries with duplicated complaint numbers.

## Find Count of Null, None, NaN of All DataFrame Columns

In [ ]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

#### Get top 5 rows where complaint from Date is NaN

In [ ]:
df.where(col('SUMMONS_DATE').isNull()).show(n=5)

+-----------+------------+-------------------+------------------+---------------+---------------------+---------+---+----+-----------------+----+-----------------+
|SUMMONS_KEY|SUMMONS_DATE|OFFENSE_DESCRIPTION|LAW_SECTION_NUMBER|LAW_DESCRIPTION|SUMMONS_CATEGORY_TYPE|AGE_GROUP|SEX|RACE|JURISDICTION_CODE|BORO|PRECINCT_OF_OCCUR|
+-----------+------------+-------------------+------------------+---------------+---------------------+---------+---+----+-----------------+----+-----------------+
+-----------+------------+-------------------+------------------+---------------+---------------------+---------+---+----+-----------------+----+-----------------+



In [ ]:
df.show(100)

+-----------+------------+--------------------+------------------+-------------------+---------------------+---------+----+--------------+-----------------+---------+-----------------+
|SUMMONS_KEY|SUMMONS_DATE| OFFENSE_DESCRIPTION|LAW_SECTION_NUMBER|    LAW_DESCRIPTION|SUMMONS_CATEGORY_TYPE|AGE_GROUP| SEX|          RACE|JURISDICTION_CODE|     BORO|PRECINCT_OF_OCCUR|
+-----------+------------+--------------------+------------------+-------------------+---------------------+---------+----+--------------+-----------------+---------+-----------------+
|  194874515|  03/19/2019|LICENSE/RECORDS/D...|           2-66(B)|               null|                 null|  UNKNOWN|null|          null|                0| NEW YORK|                1|
|  202635026|  09/19/2019|   OTHER TRAFFIC REG|              9999|                ABC|            OTHER ABC|  UNKNOWN|null|          null|                0| NEW YORK|                1|
|  143517530|  06/04/2015|CONSUMPTION OF AL...|            10-125|Administr

In [ ]:
df.agg({'SUMMONS_DATE': 'min'}).show()

+-----------------+
|min(SUMMONS_DATE)|
+-----------------+
|       01/01/2006|
+-----------------+



In [ ]:
df.agg({'SUMMONS_DATE': 'max'}).show()

+-----------------+
|max(SUMMONS_DATE)|
+-----------------+
|       12/31/2020|
+-----------------+



**Some basic data quality checks are as below:**
1. Check if there are no garbage values in the location of occurence description column. The valid values that this column should ideally contain are: 'FRONT OF', 'REAR OF', 'OUTSIDE', 'INSIDE', 'OPPOSITE OF'
2. Check if there are no garbage values in law category column. The valid values are: 'FELONY', 'VIOLATION', 'MISDEMEANOR'
3. Check if there are no misspellings in Borough Name. There should be 5 distinct boroughs: Manhattan, Bronx, Queens, Brooklyn, Staten Island. We implement unique method, in case of misspellings multiple values of the same borough would be returned.
4. Check if 'CRM_ATPT_CPTD_CD' column has no garbage value. The only acceptable values are Completed or Attempted.
5. Ideally, key code should contain only 3 digits. Implementing a check below to see if there are any invalid values for the key code. 

### Checks for Borough Name

In [ ]:
df.select('BORO').distinct().show()

+-------------+
|         BORO|
+-------------+
|        BLACK|
|            0|
|         null|
|       QUEENS|
|     BROOKLYN|
|        BRONX|
|     NEW YORK|
|    MANHATTAN|
|STATEN ISLAND|
+-------------+



We can see there are no misspellings for the Borough names and thus no need for additional data correction for the same.

In [ ]:
df.where(col('BORO').isNull()).show()

+-----------+------------+--------------------+------------------+-------------------+---------------------+---------+---+-------+-----------------+----+-----------------+
|SUMMONS_KEY|SUMMONS_DATE| OFFENSE_DESCRIPTION|LAW_SECTION_NUMBER|    LAW_DESCRIPTION|SUMMONS_CATEGORY_TYPE|AGE_GROUP|SEX|   RACE|JURISDICTION_CODE|BORO|PRECINCT_OF_OCCUR|
+-----------+------------+--------------------+------------------+-------------------+---------------------+---------+---+-------+-----------------+----+-----------------+
|   16849593|  06/17/2006|                 OCA|              null|               null|                 null|    18-24|  M|UNKNOWN|                2|null|              120|
|  143637098|  06/10/2015|UNLICENSED VEHICL...|            19-506|Administrative Code|                  TLC|    25-44|  M|   null|                0|null|              100|
|  143344009|  05/31/2015|CONSUMPTION OF AL...|            10-125|Administrative Code|              ALCOHOL|    25-44|  M|   null|          

#### Dropping Rows where Borough Name is NULL

In [ ]:
df = df.filter(df.BORO.isNotNull())

In [ ]:
df.count()

4771655

In [ ]:
df.filter(df.BORO.isNull()).show()

+-----------+------------+-------------------+------------------+---------------+---------------------+---------+---+----+-----------------+----+-----------------+
|SUMMONS_KEY|SUMMONS_DATE|OFFENSE_DESCRIPTION|LAW_SECTION_NUMBER|LAW_DESCRIPTION|SUMMONS_CATEGORY_TYPE|AGE_GROUP|SEX|RACE|JURISDICTION_CODE|BORO|PRECINCT_OF_OCCUR|
+-----------+------------+-------------------+------------------+---------------+---------------------+---------+---+----+-----------------+----+-----------------+
+-----------+------------+-------------------+------------------+---------------+---------------------+---------+---+----+-----------------+----+-----------------+



In [ ]:
df.count()

7364657

## Defining checks for outliers in age group

In [ ]:
df.select('AGE_GROUP').distinct().show()

+---------+
|AGE_GROUP|
+---------+
|      919|
|      926|
|     1008|
|      940|
|      154|
|      317|
|      938|
|      823|
|      351|
|      348|
|      924|
|      951|
|      <18|
|      813|
|      917|
|     1013|
|      933|
|      959|
|    25-44|
|      928|
+---------+
only showing top 20 rows



#### There are many invalid age groups like negative values, unrealistically high age groups, etc.

#### Lets find all the invalid age groups and replace them with `NaN`

In [ ]:
valid_age_groups = ['<18','18-24','25-44','45-64','65+',np.NaN]
df = df.withColumn('AGE_GROUP', when(df.AGE_GROUP.isin(valid_age_groups), df.AGE_GROUP).otherwise(np.NaN))
df.show()

+-----------+------------+--------------------+------------------+-------------------+---------------------+---------+----+----+-----------------+---------+-----------------+
|SUMMONS_KEY|SUMMONS_DATE| OFFENSE_DESCRIPTION|LAW_SECTION_NUMBER|    LAW_DESCRIPTION|SUMMONS_CATEGORY_TYPE|AGE_GROUP| SEX|RACE|JURISDICTION_CODE|     BORO|PRECINCT_OF_OCCUR|
+-----------+------------+--------------------+------------------+-------------------+---------------------+---------+----+----+-----------------+---------+-----------------+
|  194874515|  03/19/2019|LICENSE/RECORDS/D...|           2-66(B)|               null|                 null|      NaN|null|null|                0| NEW YORK|                1|
|  202635026|  09/19/2019|   OTHER TRAFFIC REG|              9999|                ABC|            OTHER ABC|      NaN|null|null|                0| NEW YORK|                1|
|  143517530|  06/04/2015|CONSUMPTION OF AL...|            10-125|Administrative Code|              ALCOHOL|    25-44|   M|nu

In [ ]:
df.select('AGE_GROUP').distinct().show()2

+---------+
|AGE_GROUP|
+---------+
|      <18|
|    25-44|
|      65+|
|      NaN|
|    18-24|
|    45-64|
+---------+



In [ ]:
df.show(n=5)

+-----------+------------+--------------------+------------------+-------------------+---------------------+---------+----+----+-----------------+---------+-----------------+
|SUMMONS_KEY|SUMMONS_DATE| OFFENSE_DESCRIPTION|LAW_SECTION_NUMBER|    LAW_DESCRIPTION|SUMMONS_CATEGORY_TYPE|AGE_GROUP| SEX|RACE|JURISDICTION_CODE|     BORO|PRECINCT_OF_OCCUR|
+-----------+------------+--------------------+------------------+-------------------+---------------------+---------+----+----+-----------------+---------+-----------------+
|  194874515|  03/19/2019|LICENSE/RECORDS/D...|           2-66(B)|               null|                 null|      NaN|null|null|                0| NEW YORK|                1|
|  202635026|  09/19/2019|   OTHER TRAFFIC REG|              9999|                ABC|            OTHER ABC|      NaN|null|null|                0| NEW YORK|                1|
|  143517530|  06/04/2015|CONSUMPTION OF AL...|            10-125|Administrative Code|              ALCOHOL|    25-44|   M|nu

In [ ]:
df.select('AGE_GROUP').distinct().show()

+---------+
|AGE_GROUP|
+---------+
|      <18|
|    25-44|
|      65+|
|      NaN|
|    18-24|
|    45-64|
+---------+



In [ ]:
df.show(n=5)

+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+----------------+-----------------+--------+----------+-----------+--------------+---------+--------+----------------+--------------------+--------------------+------------+-------------+--------------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|      JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|             Lat_Lon|         PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|            VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------

### Check for Race Values 

In [ ]:
df.select('RACE').distinct().show()

+--------------------+
|                RACE|
+--------------------+
|               WHITE|
|               BLACK|
|AMERICAN INDIAN/A...|
|                null|
|      BLACK HISPANIC|
|                   M|
|      WHITE HISPANIC|
|             UNKNOWN|
|               OTHER|
|               45-64|
|ASIAN / PACIFIC I...|
|            HISPANIC|
+--------------------+



#### Replace all `UNKNOWN` values with `NaN`

In [ ]:
from pyspark.sql.functions import regexp_replace

df = df.withColumn("RACE",
  regexp_replace("RACE", "UNKNOWN", "NaN"))

In [ ]:
df.show(100)

+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+-------------+-----------------+--------------------+-------------------+-----------------+--------+----------+-----------+--------------+--------------------+--------+----------------+--------------------+--------------------+------------+-------------+--------------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|      BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|         JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|           SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|             Lat_Lon|         PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|            VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+----------+-----

In [ ]:
df.select('SUSP_RACE').distinct().show()

### Checks for Suspect & Victim Sex

In [ ]:
df.select('SEX').distinct().show()

+--------+
|SUSP_SEX|
+--------+
|       F|
|    null|
|       M|
|       U|
+--------+



#### Checking values in suspect sex

In [ ]:
df.groupBy('SEX').count().orderBy('count', ascending=False).show()

+--------+-------+
|SUSP_SEX|  count|
+--------+-------+
|    null|3554679|
|       M|2390246|
|       F| 755676|
|       U| 664056|
+--------+-------+



#### Get unique values of offense description in sorted order

In [ ]:
df.select('OFFENSE_DESCRIPTION').distinct().orderBy('OFFENSE_DESCRIPTION', ascending=True).show()

+--------------------+
|           OFNS_DESC|
+--------------------+
|                null|
|            ABORTION|
| ADMINISTRATIVE CODE|
|ADMINISTRATIVE CODES|
|AGRICULTURE & MRK...|
|ALCOHOLIC BEVERAG...|
|ANTICIPATORY OFFE...|
|               ARSON|
|ASSAULT 3 & RELAT...|
|     BURGLAR'S TOOLS|
|            BURGLARY|
|CHILD ABANDONMENT...|
|CRIMINAL MISCHIEF...|
|   CRIMINAL TRESPASS|
|     DANGEROUS DRUGS|
|   DANGEROUS WEAPONS|
|  DISORDERLY CONDUCT|
|DISRUPTION OF A R...|
|ENDAN WELFARE INCOMP|
|            ESCAPE 3|
+--------------------+
only showing top 20 rows



#### Getting Total Count of Offense Description

In [ ]:
df.select('OFFENSE_DESCRIPTION').distinct().count()

71

In [ ]:
df.groupBy('OFFENSE_DESCRIPTION').count().show()

+--------------------+------+
|           OFNS_DESC| count|
+--------------------+------+
|ANTICIPATORY OFFE...|   128|
|   FELONY SEX CRIMES|    18|
|NEW YORK CITY HEA...|    93|
|OTHER OFFENSES RE...| 16226|
|VEHICLE AND TRAFF...| 91734|
|KIDNAPPING & RELA...|  2874|
|HOMICIDE-NEGLIGEN...|   104|
|OFF. AGNST PUB OR...|383063|
|PETIT LARCENY OF ...|  1334|
|      FELONY ASSAULT|285879|
|OFFENSES RELATED ...|  1691|
|ALCOHOLIC BEVERAG...|  1277|
|CRIMINAL MISCHIEF...|744482|
|         THEFT-FRAUD| 76591|
|   THEFT OF SERVICES|  4527|
|            JOSTLING|   446|
|MISCELLANEOUS PEN...|185656|
|LOITERING/GAMBLIN...|   245|
|               ARSON| 17639|
|OFFENSES AGAINST ...| 18157|
+--------------------+------+
only showing top 20 rows



#### Each key code represents a particular offense description. There is a one to one mapping. So we would use key code for future analysis instead of offense description.

#### Calculating the null values present in the data columnwise (with respect to the features)

In [ ]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

#### Thus, we can see that the percentage of null values per variable has gone considerably down after cleaning. Some variables like 'PARKS_NM', 'HADEVELOPT' and such can have null values as established above. 

In [ ]:
df.groupBy('JURISDICTION_CODE').count().show()

### Number of columns in Clean Data

In [ ]:
len(df.columns)

### Number of rows in Clean Data

In [ ]:
df.count()

In [ ]:
df.printSchema()

### **Exporting Clean Data in CSV**

The Cleaned Data Set will be saved as `NYPD_Complaint_Data_Historic_Cleaned.csv`

In [ ]:
pd_df = df.toPandas()
pd_df.to_csv("NYPD_Criminal_Court_Summons_Cleaned_Spark.csv")